In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

In [2]:
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

Request c3dd6624-8417-4e3f-94ae-75fcb477b8b6: HEAD https://hf-mirror.com/facebook/opt-6.7b/resolve/main/config.json (authenticated: False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Request 844bf116-e5b8-47dd-b0b9-2113a7be0e9b: HEAD https://hf-mirror.com/facebook/opt-6.7b/resolve/main/generation_config.json (authenticated: False)
Request c8af405f-27a6-4e4f-a9e5-7e4d5e45c509: HEAD https://hf-mirror.com/facebook/opt-6.7b/resolve/main/tokenizer_config.json (authenticated: False)
Request baa593ef-a8cd-40eb-b8b8-62757e4f3dbf: GET https://hf-mirror.com/api/datasets/wikitext (authenticated: False)
Request 4a836e26-ac44-44fc-b833-beb165a58dca: GET https://hf-mirror.com/api/datasets/wikitext (authenticated: False)
Request 057e9529-8e2a-4485-bae8-6ac141c5bbec: GET https://hf-mirror.com/api/datasets/wikitext/revision/b08601e04326c79dfdd32d625aee71d232d685c3 (authenticated: False)
Request ab7a30fa-e0c8-4338-9330-54c86c788888: POST https://hf-mirror.com/api/datasets/wikitext/paths-info/b08601e04326c79dfdd32d625aee71d232d685c3 (authenticated: False)
Request f26952fe-820e-47f3-88d3-49f3ea9d914f: POST https://hf-mirror.com/api/datasets/wikitext/paths-info/b08601e04326c79dfdd32d62

Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/root/miniconda3/envs/glm/lib/python3.10/site-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

In [ ]:
# 保存模型权重
quant_model.save_pretrained("models/opt-6.7b-gptq")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_name_or_path = "facebook/opt-2.7b"
custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

In [ ]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))